In [1]:
import pandas as pd
import json
import requests
import math
import numpy as np

In [2]:
from censusAPI import myAPI
import utilcalcs as calc
import geo_agg
from muni_geo_id import PUMA_2018,cousub_2018,placeLI_2018,\
                        cousub_2010,placeLI_2010,stco,state

## User-defined parameters

In [3]:
nyc = ['005','047','061','081','085']

In [4]:
#Variables and predicates for both start/end years
#broadband access
source = 'acs/acs5'

col_b = f'GEO_ID,B28002_001E,B28002_001M,B28002_002E,B28002_002M,B28002_004E,B28002_004M,B28002_013E,B28002_013M,B28003_006E,B28003_006M' 
col_d = f'GEO_ID,DP02_0001E,DP02_0001M,DP02_0151E,DP02_0151M,DP02_0152E,DP02_0152M' 

year1 = '2018'
#year0 = '2010'

var_data = ['GEO_ID','B28002_001E','B28002_001M','B28002_002E','B28002_002M','B28002_004E','B28002_004M','B28002_013E','B28002_013M','B28003_006E',\
            'B28003_006M','DP02_0001E','DP02_0001M','DP02_0151E','DP02_0151M','DP02_0152E','DP02_0152M']

### Data download functions

In [5]:
def get_cousub(year,col_b,col_d):#,geo_code):
    frames1 = []
    frames2 = []
    for st,co in stco.items():
        for i in co:
            url = f'https://api.census.gov/data/{year}/{source}?get={col_b}&for=county%20subdivision:*&in=state:{st}%20county:{i}&key={myAPI}'
            resp = requests.request('GET', url).content
            df = pd.DataFrame(json.loads(resp)[1:])
            df.columns = json.loads(resp)[0]
            frames1.append(df)
    for st,co in stco.items():
        for i in co:
            url = f'https://api.census.gov/data/{year}/{source}/profile?get={col_d}&for=county%20subdivision:*&in=state:{st}%20county:{i}&key={myAPI}'
            resp = requests.request('GET', url).content
            df = pd.DataFrame(json.loads(resp)[1:])
            df.columns = json.loads(resp)[0]
            frames2.append(df)
    df_sub = pd.merge(pd.concat(frames1),pd.concat(frames2),how='left',on='GEO_ID')
    #df_sub = df_sub[df_sub['GEO_ID'].isin(geo_code)]
    return df_sub

def get_place(year,col_b,col_d,geo_code):
    url1 = f'https://api.census.gov/data/{year}/{source}?get={col_b}&for=place:*&in=state:36&key={myAPI}'
    resp1 = requests.request('GET', url1).content
    df1 = pd.DataFrame(json.loads(resp1)[1:])
    df1.columns = json.loads(resp1)[0]
    url2 = f'https://api.census.gov/data/{year}/{source}/profile?get={col_d}&for=place:*&in=state:36&key={myAPI}'
    resp2 = requests.request('GET', url2).content
    df2 = pd.DataFrame(json.loads(resp2)[1:])
    df2.columns = json.loads(resp2)[0]
    
    df_pl = pd.merge(df1,df2,how='left',on='GEO_ID')
    df_pl = df_pl[df_pl['GEO_ID'].isin(geo_code)]
    return df_pl

def get_nyc_tract(year,col_b,col_d):
    frames1 = []
    frames2 = []
    for i in nyc:
        url1 = f'https://api.census.gov/data/{year}/{source}?get={col_b}&for=tract:*&in=state:36&in=county:{i}&key={myAPI}'
        resp1 = requests.request('GET', url1).content
        df1 = pd.DataFrame(json.loads(resp1)[1:])
        df1.columns = json.loads(resp1)[0]
        frames1.append(df1)
    for i in nyc:
        url2 = f'https://api.census.gov/data/{year}/{source}/profile?get={col_d}&for=tract:*&in=state:36&in=county:{i}&key={myAPI}'
        resp2 = requests.request('GET', url2).content
        df2 = pd.DataFrame(json.loads(resp2)[1:])
        df2.columns = json.loads(resp2)[0]
        frames2.append(df2)
    df_tract = pd.merge(pd.concat(frames1),pd.concat(frames2),how='left',on='GEO_ID')
    return df_tract

def clean_data(df,var):
    dff = df[var].copy()
    var_num = var[1:]
    for col in var_num:
        dff[col] = dff[col].astype(float)
    dff = dff.replace([999999999, 555555555, 333333333, 222222222,\
                    666666666, 888888888, -999999999, -555555555,\
                    -333333333, -222222222, -666666666, -888888888], np.nan)
    return dff


### Variables for table calculations

## Households and Broadband Year 1

#### Subdivisions in NY-NJ-CT - Places in LI

In [6]:
dfY1_sub = get_cousub(year1,col_b,col_d)#,cousub_2018)
dfY1_pl = get_place(year1,col_b,col_d,placeLI_2018)
dfY1 = pd.concat([dfY1_sub,dfY1_pl],sort=True)
dfY1 = clean_data(dfY1,var_data)

In [7]:
dfY1.head()

,GEO_ID,B28002_001E,B28002_001M,B28002_002E,B28002_002M,B28002_004E,B28002_004M,B28002_013E,B28002_013M,B28003_006E,B28003_006M,DP02_0001E,DP02_0001M,DP02_0151E,DP02_0151M,DP02_0152E,DP02_0152M
0,0600000US0900133620,22251.0,380.0,20482.0,415.0,20404.0,424.0,1359.0,212.0,1137.0,188.0,22251.0,380.0,21114.0,362.0,20404.0,424.0
1,0600000US0900148620,6703.0,223.0,5907.0,288.0,5891.0,291.0,541.0,198.0,487.0,169.0,6703.0,223.0,6216.0,238.0,5891.0,291.0
2,0600000US0900163480,3440.0,160.0,3159.0,164.0,3120.0,159.0,136.0,68.0,118.0,58.0,3440.0,160.0,3322.0,160.0,3120.0,159.0
3,0600000US0900174190,20084.0,414.0,16180.0,536.0,16116.0,530.0,3038.0,411.0,2449.0,370.0,20084.0,414.0,17635.0,516.0,16116.0,530.0
4,0600000US0900150580,7084.0,186.0,6689.0,225.0,6689.0,225.0,307.0,130.0,273.0,128.0,7084.0,186.0,6811.0,227.0,6689.0,225.0


#### Tracts for NYC Only - to calculate as NTAs

In [8]:
dfY1_nyc = get_nyc_tract(year1,col_b,col_d)
dfY1_nyc = clean_data(dfY1_nyc,var_data)

In [9]:
dfY1_nyc['join_id'] = dfY1_nyc['GEO_ID'].str[9:].astype(int)

In [10]:
#import csv to recode tracts to NTAs
geo = pd.read_csv('data/nyc_xgeo.csv') 

In [11]:
dfY1_nyc = pd.merge(dfY1_nyc, geo, left_on="join_id", right_on="tract", how="inner").drop(columns=['tract','GEO_ID','join_id','PUMA','NTA_name','Subbor_name','Subbor'])

In [12]:
dfY1_nyc.head()

,B28002_001E,B28002_001M,B28002_002E,B28002_002M,B28002_004E,B28002_004M,B28002_013E,B28002_013M,B28003_006E,B28003_006M,DP02_0001E,DP02_0001M,DP02_0151E,DP02_0151M,DP02_0152E,DP02_0152M,NTA
0,1599.0,67.0,1169.0,135.0,1158.0,134.0,419.0,125.0,324.0,100.0,1599.0,67.0,1275.0,114.0,1158.0,134.0,BX43
1,2129.0,104.0,1626.0,202.0,1582.0,204.0,407.0,197.0,323.0,154.0,2129.0,104.0,1806.0,187.0,1582.0,204.0,BX07
2,2443.0,122.0,1979.0,192.0,1979.0,192.0,245.0,135.0,176.0,101.0,2443.0,122.0,2267.0,142.0,1979.0,192.0,BX03
3,1739.0,107.0,1021.0,140.0,1021.0,140.0,628.0,148.0,456.0,133.0,1739.0,107.0,1283.0,139.0,1021.0,140.0,BX17
4,1674.0,83.0,1070.0,136.0,1070.0,136.0,470.0,116.0,365.0,105.0,1674.0,83.0,1309.0,128.0,1070.0,136.0,BX01


In [13]:
cols = list(dfY1_nyc.columns)
cols = [cols[-1]]+cols[:-1]
dfY1_nyc = dfY1_nyc[cols]

In [14]:
#Aggregate pumas to sub-borough geos & calc MOEs
dfY1_nyc = geo_agg.calc_muni_agg(dfY1_nyc,'NTA')
dfY1_nyc = dfY1_nyc.rename(columns={'NTA':'GEO_ID'})
dfY1_nyc.head()

,GEO_ID,DP02_0151E,DP02_0151M,B28002_001E,B28002_001M,B28002_002E,B28002_002M,B28003_006E,B28003_006M,B28002_013E,B28002_013M,DP02_0152E,DP02_0152M,DP02_0001E,DP02_0001M,B28002_004E,B28002_004M
0,BX43,12058.0,462.576480,14505.0,301.469733,10645.0,524.828543,2447.0,386.283575,2964.0,431.238913,10615.0,525.646269,14505.0,301.469733,10615.0,525.646269
1,BX07,12247.0,363.964284,14562.0,230.130398,10785.0,451.151859,2315.0,324.930762,2860.0,389.991026,10673.0,450.879141,14562.0,230.130398,10673.0,450.879141
2,BX03,10432.0,371.113190,12038.0,231.090891,9055.0,454.840631,1606.0,309.935477,2253.0,373.436742,9019.0,456.275136,12038.0,231.090891,9019.0,456.275136
3,BX17,12414.0,386.732466,15289.0,272.794795,9627.0,478.801629,2875.0,332.305582,4916.0,446.832183,9590.0,481.102900,15289.0,272.794795,9590.0,481.102900
4,BX01,8782.0,349.912846,11131.0,181.934054,7118.0,369.997297,2349.0,311.929479,3125.0,351.917604,7118.0,369.997297,11131.0,181.934054,7118.0,369.997297


#### Combine SubPlace and NYCPUMA Table into Municipality Table for 2018

In [15]:
dfY1 = pd.concat([dfY1,dfY1_nyc],sort=True)
dfY1.head()

,B28002_001E,B28002_001M,B28002_002E,B28002_002M,B28002_004E,B28002_004M,B28002_013E,B28002_013M,B28003_006E,B28003_006M,DP02_0001E,DP02_0001M,DP02_0151E,DP02_0151M,DP02_0152E,DP02_0152M,GEO_ID
0,22251.0,380.0,20482.0,415.0,20404.0,424.0,1359.0,212.0,1137.0,188.0,22251.0,380.0,21114.0,362.0,20404.0,424.0,0600000US0900133620
1,6703.0,223.0,5907.0,288.0,5891.0,291.0,541.0,198.0,487.0,169.0,6703.0,223.0,6216.0,238.0,5891.0,291.0,0600000US0900148620
2,3440.0,160.0,3159.0,164.0,3120.0,159.0,136.0,68.0,118.0,58.0,3440.0,160.0,3322.0,160.0,3120.0,159.0,0600000US0900163480
3,20084.0,414.0,16180.0,536.0,16116.0,530.0,3038.0,411.0,2449.0,370.0,20084.0,414.0,17635.0,516.0,16116.0,530.0,0600000US0900174190
4,7084.0,186.0,6689.0,225.0,6689.0,225.0,307.0,130.0,273.0,128.0,7084.0,186.0,6811.0,227.0,6689.0,225.0,0600000US0900150580


In [16]:
dfY1.to_excel('output/Broadband/subplNTA_broadband.xlsx')